In [1]:
%load_ext autoreload

In [108]:
%autoreload 2

In [3]:
from gymnasium import Env, spaces, register, make
import numpy as np
import random

In [109]:
from RME_file import RandomMazeEnvironment as RM_env
from RME_file import Possibilities as P

In [110]:
register(id = 'RME-mark0', entry_point=RM_env)

C:\Users\hp\Desktop\CS780_Assignment1\cs780_env\lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment RME-mark0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [111]:
env = make('RME-mark0')

In [87]:
theta = 10 ** -10
gamma = 0.99
totalStates = 12

In [8]:
real_state_array = [x for x in range(12) if x != 5]
real_state_array
# since my env has no possibility of state 5 defined, 
# I make a range of 0 till 11 excluding 5

[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11]

In [9]:
# assuming a general policy, i.e. for each state, their is a differnt probability of taking any action out of the action space, and not that for every state, 
# there is the same probability of taking an action.
# example
# This is taken: policy[state = 1] = [0.1, 0.1, 0.1, 0.7] and also policy[state = 2] = [0.2, 0.3,0.0,0.5]
# This could be the case but NOT always true: policy[state = 1] = policy[state = 2] = [0.1, 0.1, 0.1, 0.7] 

# Subproblem 1

In [30]:
def PolicyEvaluation(policy, P, gamma, theta):
    v_old = np.zeros(totalStates) # value of state 5 will never change from 0
    
    while True:
        v_new = np.zeros(totalStates) # value of state 5 will never change from 0
        for s in real_state_array:
            for a in range(4):
                temp = 0
                for prob, s_next, reward, termination in P[s][a]:
                    temp += prob*(reward + gamma*v_old[s_next])
                if policy[s] == a:
                    v_new[s] += 1*temp
                else:
                    v_new[s] += 0*temp
        if abs(np.amax(v_new)-np.amax(v_old)) < theta:
            break
        else:
            v_old = v_new
    return v_new

In [31]:
def PolicyImprovement(v, P, gamma):
    Q = np.zeros((totalStates, 4)) #total 4 actions in RME
    policy = np.zeros(totalStates)
    
    for s in real_state_array:
        for a in range(4):
            for prob, s_next, reward, termination in P[s][a]:
                Q[s][a] += prob*(reward + gamma* v[s_next])
                
    for s in real_state_array: #even if we include terminal state while updating policy,no diff as reward is 0 after they reach terminal state
        policy[s] = np.argmax(Q[s]) #best action for state s
    
    #return best action for all states
    return policy
        

In [32]:
def PolicyIteration(P, gamma, theta, initial_policy):
    # choosing a policy to always take 0 action, meaning always go up, as initial random policy
    policy = initial_policy
    
    no_iterations = 0
    while True:
        no_iterations += 1
        policy_old = policy
        v = PolicyEvaluation(policy, P, gamma, theta)
        policy = PolicyImprovement(v, P, gamma)        
        if np.array_equal(policy_old, policy):
            break
    
    return v, policy, no_iterations

In [70]:
initial_policy = np.array([1,2,3,0,1,2,3,0,1,2,3,0])
initial_policy

array([1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0])

In [106]:
v, policy, no_iterations = PolicyIteration(P, gamma, theta, initial_policy)
np.random.seed(9569)
print('Policy Iteration:')
print(f'Initial Policy: {initial_policy}\nOptimal Policy: {policy}\nNumber of iterations: {no_iterations}')

Policy Iteration:
Initial Policy: [1 2 3 0 1 2 3 0 1 2 3 0]
Optimal Policy: [1. 1. 1. 0. 0. 0. 0. 0. 0. 3. 0. 3.]
Number of iterations: 3


# SubProblem 2

In [88]:
def ValueIteration(P,gamma,theta, initial_policy):
    policy = initial_policy
    v = np.zeros(totalStates)
    iterations=0
    
    while True:
        iterations += 1
        Q=np.zeros((totalStates, 4))
        for s in real_state_array:
            for a in range(4):
                for prob, s_next, reward, termination in P[s][a]:
                    Q[s][a] += prob*(reward + gamma* v[s_next])
                    
        breaking_point = 1
        
        # if max value is smaller that theta, all the values must be smaller than theta,
        # thus if any value is less than theta, we give break
        for s in real_state_array:
            if abs(v[s]-np.amax(Q[s])) >= theta:
                breaking_point =  0
                break
                
        if breaking_point == 1:
            break
            
        for s in real_state_array:
            v[s]=np.amax(Q[s])
        
        for s in real_state_array:
            policy[s]=np.argmax(Q[s])
            
    return v, policy, iterations  

In [103]:
# initial_policy = np.array([1,2,3,0,1,2,3,0,1,2,3,0])

In [107]:
v, optimal_policy, no_iterations = ValueIteration(P, gamma, theta, np.array([1,2,3,0,1,2,3,0,1,2,3,0]))
np.random.seed(9569)
print('Value Iteration:')
print(f'Initial Policy: {initial_policy}\nOptimal Policy: {optimal_policy}\nNumber of iterations: {no_iterations}')

Value Iteration:
Initial Policy: [1 2 3 0 1 2 3 0 1 2 3 0]
Optimal Policy: [1 1 1 0 0 2 0 0 0 3 0 3]
Number of iterations: 38
